In [1]:
import warnings
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import plotly.express as px

from constants import *
from preprocessor import Preprocessor
from utils.utilss import customer_cumsum, monthly_revenue_and_transactions,\
    country_amount, top_countries_by_month_year, get_whale_customers, \
        get_whales_by_column
from fe import FeatureEngineering 
from model import ClusterModel




ModuleNotFoundError: No module named 'constants'

In [ ]:
# for readability

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 99999)
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_parquet('df.parquet.gzip')
CURRENT_TIME = df[invoicedate].max()

In [6]:
df.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df[invoicedate].min(), df[invoicedate].max()

(Timestamp('2010-12-01 08:26:00'), Timestamp('2011-12-09 12:50:00'))

cleaning

In [ ]:
preprocessor = Preprocessor()
processed_df = preprocessor.preprocessing_pipeline(df)
print(f'total customers - {processed_df[customerid].nunique()}')

In [ ]:
customer_amount = processed_df.groupby(customerid, as_index=False)[amount].sum()
customer_amount['bin_total_amount'] = pd.qcut(customer_amount[amount], 10)
series_vc = customer_amount['bin_total_amount']
pd.concat([series_vc.value_counts(dropna=False), series_vc
                .value_counts(normalize=True, dropna=False)], keys=['counts', 'normalized_counts'], axis=1)



In [ ]:
customer_cumsum_df = customer_cumsum(processed_df)
fig = px.line(customer_cumsum_df, x=range(customer_cumsum_df.shape[0]), y="amount_cumsum",
             title='accumaulated revenue with respect to total customers'.title())
fig.update_layout(xaxis_title="accumulated customers".title(), yaxis_title="amount ratio".title())
fig.show()

from the above plot we can see sort of the pareto rule - about 500 customers (12.5%) are responsible for 60% of the revenue

aa

In [ ]:
monthly_amount, monthly_transactions, monthly_avg_transaction_amount = monthly_revenue_and_transactions(processed_df)

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(12, 10))
sns.lineplot(monthly_amount, x="month_year", y='amount', ax=ax1).set_title('month with respect to total revenue'.title())
sns.lineplot(monthly_transactions, x="month_year", y=invoiceno, ax=ax2).set_title('total transactions per month'.title())
sns.lineplot(monthly_avg_transaction_amount, x="month_year", y='avg_transaction_amount', ax=ax3).set_title('average amount per invoice'.title())

plt.tight_layout()
plt.show()

In [ ]:
# dead customers

processed_df['max_date'] = processed_df.groupby(customerid)[invoicedate].transform('max')
processed_df['recency'] = (CURRENT_TIME - processed_df['max_date']).dt.days
dead_customer = processed_df.groupby([customerid, 'recency'], as_index=False)[invoiceno].nunique()
th_dead_customer = 180
mask = (dead_customer['recency'] > th_dead_customer) & (dead_customer[invoiceno] == 1)
dead_customer[mask].shape


In [ ]:
# explore top countries by amount

country_df = country_amount(processed_df, N=processed_df[country].nunique())
country_df[:10]


we can see from the above table that 5 countries are responsibe for 95% of the income.<br>
More importantly, we can that the UK has the majority of out bussiness, with 83% of out total income.<br>
let's visualize it for better readability.

In [ ]:
# top n countries by amount

n = 5
country_df = country_amount(processed_df, N=n)
top_countries = country_df[:n][country]
title = '95% of the income is coming from the following 5 countries  (83% is coming from the UK)'.title()

fig = px.bar(country_df, x=country, y='amount_percent', text='amount_percent', title=title, color = country
             ).update_layout(showlegend=False, yaxis_range=[0,1]) 
fig.show()

In [ ]:
# time series per top countries

# month_year = 'month_year'
country_month_year_top = top_countries_by_month_year(processed_df, top_countries)

fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1, sharex=True, figsize=(14, 12))
axs = [ax1, ax2, ax3, ax4, ax5]
for i, (ax, country_name) in enumerate(zip(axs, top_countries)):
    i+=1
    country_df_temp = country_month_year_top[country_month_year_top[country] == country_name]
    plt.subplot(n*100 + 10 + i)
    sns.lineplot(country_df_temp, x='month_year', y=amount, ax=ax).set_title(f'{country_name}'.upper())
plt.tight_layout()

In [ ]:
# whales

whales = get_whale_customers(processed_df, q=.95)
print(f'there are total {whales.shape[0]} whale customers')
whales_count = get_whales_by_column(whales, groupby_column=country, top_countries_n=5)
fig = px.bar(whales_count, x=country, y='customers', title='Whales per country'.title(), color = country,
             text='customers'). update_layout(showlegend=False)
fig.show()

In [ ]:
# total transactions per country - time series


month_year_country_rans = processed_df.groupby(['month_year', country], as_index=False)[invoiceno].size().rename(
    columns={'size': 'transactions'})
month_year_country_rans[month_year] = month_year_country_rans[month_year].astype(str)
month_year_country_rans = month_year_country_rans[month_year_country_rans[country].isin(top_countries)]

fig, (ax1, ax2, ax3,ax4,ax5) = plt.subplots(n, 1, sharex=True, figsize=(14, 12))
axs = [ax1, ax2, ax3,ax4,ax5]
for i, (ax, country_name) in enumerate(zip(axs, top_countries)):
    i+=1
    month_year_country_rans_temp = month_year_country_rans[month_year_country_rans[country] == country_name]
    plt.subplot(n*100 + 10 + i)
    sns.lineplot(month_year_country_rans_temp, x='month_year', y='transactions', ax=ax
                 ).set_title(f'{country_name}'.upper())
plt.tight_layout()

In [ ]:
# new customers per month

processed_df['join_date'] = processed_df.groupby(customerid)[invoicedate].transform('min').dt.to_period(freq='M')

join_df = processed_df[['join_date', customerid]].drop_duplicates()
join_df = join_df.groupby('join_date', as_index=False)[customerid].nunique().rename(
    columns={customerid: 'num_of_new_customers'})
join_df['join_date'] = join_df['join_date'].astype(str)

figure(figsize=(16, 6), dpi=80)
title = f'new customers with respect to month'.title()
sns.lineplot(join_df, x='join_date', y='num_of_new_customers').set_title(title)
plt.tight_layout()

In [ ]:
# new customers per month by country

processed_df['join_date'] = processed_df.groupby(customerid)[invoicedate].transform('min').dt.to_period(freq='M')

join_df = processed_df[['join_date', customerid, country]].drop_duplicates()
join_df = join_df.groupby(['join_date', country], as_index=False)[customerid].nunique().rename(
    columns={customerid: 'num_of_new_customers'})
join_df['join_date'] = join_df['join_date'].astype(str)
join_df = join_df[join_df[country].isin(top_countries)]
join_df[:2]

i = 0
fig, (ax1, ax2, ax3,ax4,ax5) = plt.subplots(n, 1, sharex=True, figsize=(14, 12))
axs = [ax1, ax2, ax3,ax4,ax5]
for ax, c in zip(axs, top_countries):
    i+=1
    cc = join_df[join_df[country] == c]
    plt.subplot(n*100 + 10 + i)
    sns.lineplot(cc, x='join_date', y='num_of_new_customers', ax=ax).set_title(f'{c}'.upper())
plt.tight_layout()

In [ ]:
# plot recency distribution to see active customers

### EDA main insights

## Feature Engineering

In [ ]:
fe = FeatureEngineering()
fe_df = fe.feature_engineering_pipeline(processed_df)
print(f'total customers after applying feature engineering - {processed_df[customerid].nunique()}')

In [ ]:
fe_df.head()

In [ ]:
# box plots

In [ ]:
# normalizing



mms = MinMaxScaler()
fe_df_customers = fe_df[customerid]
fe_df = fe_df.drop([customerid], axis=1)
X = pd.DataFrame(fe_df, columns=[i for i in fe_df.columns])
X = mms.fit_transform(X)

### elbow method

In [ ]:
k_means=ClusterModel(X, elbow_max_clusters=10, model=KMeans)
k_means.elbow_method()
k_means.plot_elbow()

In [ ]:
n_clusters = 2
hyper_params = {'n_clusters': n_clusters, 'random_state': 1}
k_means = ClusterModel(X, elbow_max_clusters=10, model=KMeans, model_hyperparameters=hyper_params)
k_means.fit_model()
fe_df['Cluster'] = k_means.fitted_model.fit_predict(X=k_means.X)
fe_df['Cluster'] += 1

In [ ]:
fe_df['Cluster'] = fe_df['Cluster'].astype(str)

fig = px.scatter_3d(fe_df, x='recency', y='frequency', z='monetary_value',
                    color='Cluster', title='3D Plot of K-means Clusters on RFM Data'
                    ,width=800, height=800).update_layout(showlegend=False)
fig.show()

let's remove the outliers for better visibility

In [ ]:
plot_df = fe_df[fe_df['frequency'] < 50]
plot_df = fe_df[fe_df['monetary_value'] < 100_000]

fig = px.scatter_3d(plot_df, x='recency', y='frequency', z='monetary_value',
                    color='Cluster', title='3D Plot of K-means Clusters on RFM Data'
                    ,width=800, height=800).update_layout(showlegend=False)
fig.show()

PCA

In [ ]:
pca = PCA(n_components=5)
pca.fit(X)
print(f'explained_variance_ratio - {pca.explained_variance_ratio_}')


In [ ]:

pca_components = PCA(n_components=2)
pca_components = pca.fit_transform(X)

fe_df['PCA1'] = pca_components[:, 0]
fe_df['PCA2'] = pca_components[:, 1]

# Plot the first two PCA components with different colors for each cluster
plt.figure(figsize=(10, 7))
colors = ['r', 'g', 'b', 'c', 'm']
for cluster in range(n_clusters):
    clustered_data = fe_df[fe_df['Cluster'] == str(cluster+ 1)]
    plt.scatter(clustered_data['PCA1'],
                clustered_data['PCA2'],
                c=colors[cluster],
                label=f'Cluster {cluster+1}')

plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.title('PCA Colored by Cluster')
plt.legend()
plt.show()

Clusters Evaluation

In [ ]:
print(fe_df.shape)
fe_df_full = pd.concat([fe_df, fe_df_customers], axis=1)
print(fe_df_full.shape)

cluster4 = fe_df_full[fe_df_full['Cluster'] == '4']
cluster3 = fe_df_full[fe_df_full['Cluster'] == '3']
cluster2 = fe_df_full[fe_df_full['Cluster'] == '2']
cluster1 = fe_df_full[fe_df_full['Cluster'] == '1']

print(cluster4[cluster4[customerid].isin(whales[customerid])].shape[0])
print(cluster3[cluster3[customerid].isin(whales[customerid])].shape[0])
print(cluster2[cluster2[customerid].isin(whales[customerid])].shape[0])
print(cluster1[cluster1[customerid].isin(whales[customerid])].shape[0])
print(whales.shape[0])
cluster4.shape,cluster3.shape, cluster2.shape, cluster1.shape